In [1]:
import tensorflow as tf
from keras.utils import np_utils
from keras.models import load_model
import keras.backend as K
import random 
import numpy as np
import pandas as pd

from preprocessor import _imread as imread
from preprocessor import _imresize as imresize
from matplotlib import pyplot as plt
from keras.losses import categorical_crossentropy
import argparse
import glob
import os

Using TensorFlow backend.


In [2]:
DATA_DIR = "data/UTKFace"
TRAIN_TEST_SPLIT = 0.7
IM_WIDTH = IM_HEIGHT = 198
input_shape = (IM_WIDTH, IM_HEIGHT, 3)
ID_GENDER_MAP = {0: 'male', 1: 'female'}
GENDER_ID_MAP = dict((g, i) for i, g in ID_GENDER_MAP.items())
ID_RACE_MAP = {0: 'white', 1: 'black', 2: 'asian', 3: 'indian', 4: 'others'}
RACE_ID_MAP = dict((r, i) for i, r in ID_RACE_MAP.items())

ID_GENDER_MAP, GENDER_ID_MAP, ID_RACE_MAP, RACE_ID_MAP

({0: 'male', 1: 'female'},
 {'female': 1, 'male': 0},
 {0: 'white', 1: 'black', 2: 'asian', 3: 'indian', 4: 'others'},
 {'asian': 2, 'black': 1, 'indian': 3, 'others': 4, 'white': 0})

In [3]:
def parse_filepath(filepath):
    try:
        path, filename = os.path.split(filepath)
        filename, ext = os.path.splitext(filename)
        age, gender, race, _ = filename.split("_")
        return int(age), ID_GENDER_MAP[int(gender)], ID_RACE_MAP[int(race)]
    except Exception as e:
        print(filepath)
        return None, None, None

In [4]:
# create a pandas data frame of images, age, gender and race
files = glob.glob(os.path.join(DATA_DIR, "*.jpg"))
attributes = list(map(parse_filepath, files))

data/UTKFace/61_1_20170109142408075.jpg.chip.jpg
data/UTKFace/39_1_20170116174525125.jpg.chip.jpg
data/UTKFace/61_1_20170109150557335.jpg.chip.jpg


In [5]:
df = pd.DataFrame(attributes)
df['file'] = files
df.columns = ['age', 'gender', 'race', 'file']
df = df.dropna()
df = df[(df['age'] > 10) & (df['age'] < 65)]
df.head()

,age,gender,race,file
0,28.0,female,white,data/UTKFace/28_1_0_20170116164219746.jpg.chip...
1,24.0,female,white,data/UTKFace/24_1_0_20170117150731090.jpg.chip...
3,24.0,female,black,data/UTKFace/24_1_1_20170113003752421.jpg.chip...
4,26.0,female,indian,data/UTKFace/26_1_3_20170104235148954.jpg.chip...
5,51.0,male,white,data/UTKFace/51_0_0_20170117190825002.jpg.chip...


In [6]:
def preprocess_image(image, input_shape):
    image = imresize(image, input_shape[:2])
    image = image.astype('float32')
    image = image/255.0
    image = np.expand_dims(image, axis=0)
    return image

def predict(model, image):
    predictions = model.predict(image)
    return predictions

def FGSM(x, model):
    sess = K.get_session()
    x_adv1 = x
    x_adv2 = x
    x_adv3 = x
    x_adv4 = x
    x_adv5 = x
    alpha_1 = 1.
    alpha_2 = -1.
    #print(list(map(lambda x: x.name, model.layers)))
    # dense7 -> layer before race output
    # dense8 -> layer before gender output
    dense_7 = model.get_layer('dense_7').output
    dense_7_grads = K.gradients(dense_7, model.input)

    dense_8 = model.get_layer('dense_8').output
    dense_8_grads = K.gradients(dense_8, model.input)

    final_grads = tf.constant(alpha_1) * dense_7_grads + tf.constant(alpha_2) * dense_8_grads
    #grads = K.gradients(final_grads, model.input)
    delta = K.sign(final_grads[0])
    x_adv1 = x_adv1 + 0.1 * delta
    x_adv2 = x_adv2 + 0.2 * delta
    x_adv3 = x_adv3 + 0.3 * delta
    x_adv4 = x_adv4 + 0.4 * delta
    x_adv5 = x_adv5 + 0.5 * delta
    
    x_adv1 = K.clip(x_adv1, 0.0 ,1.0)
    x_adv2 = K.clip(x_adv2, 0.0 ,1.0)
    x_adv3 = K.clip(x_adv3, 0.0 ,1.0)
    x_adv4 = K.clip(x_adv4, 0.0 ,1.0)
    x_adv5 = K.clip(x_adv5, 0.0 ,1.0)

    gradients, x_adv1_array, x_adv2_array, x_adv3_array, x_adv4_array, x_adv5_array = sess.run([final_grads,
                                                                                                x_adv1,
                                                                                                x_adv2,
                                                                                                x_adv3,
                                                                                                x_adv4,
                                                                                                x_adv5],
                                                                                                feed_dict={model.input:x})
    #print('GRADIENT SUM:{}'.format(np.sum(gradients[0])))
    return x_adv1_array, x_adv2_array, x_adv3_array, x_adv4_array, x_adv5_array

def plot_adversarial(img_list):
    plt.figure(figsize=(8,8))
    eps = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
    for n, img in enumerate(img_list):
        ax = plt.subplot(2,3,n+1)
        ax.set_title('Epsilon: {}'.format(eps[n]))
        plt.imshow(img[0])
        plt.grid(False)
        plt.xticks([])
        plt.yticks([])
    plt.show()

In [7]:
adv_model_path = "./models/VGG16_adv_model.h5"
clean_model_path = "./models/VGG16_clean_model.h5"

In [8]:
def get_adv_samples(x, model):
    adv_sample = FGSM(x, model)

In [9]:
def save_all_adv_samples(generator, model):
    folder = "adv_generated_imgs"
    for data in generator:
        samples, labels, filenames = data
        # running loop over a batch
        prediction = predict(model, samples)
        adv_samples = FGSM(samples, model)
        for index in range(len(samples)):
            filename = filenames[index]

            final_name = "{}/{}/{}/{}".format(filename.split("/")[0], folder, "1", filename.split("/")[-1])
            img = Image.fromarray(np.uint8(adv_samples[0][index]*255))
            img.save(final_name)

            final_name = "{}/{}/{}/{}".format(filename.split("/")[0], folder, "2", filename.split("/")[-1])
            img = Image.fromarray(np.uint8(adv_samples[1][index]*255))
            img.save(final_name)

            final_name = "{}/{}/{}/{}".format(filename.split("/")[0], folder, "3", filename.split("/")[-1])
            img = Image.fromarray(np.uint8(adv_samples[2][index]*255))
            img.save(final_name)

            final_name = "{}/{}/{}/{}".format(filename.split("/")[0], folder, "4", filename.split("/")[-1])
            img = Image.fromarray(np.uint8(adv_samples[3][index]*255))
            img.save(final_name)

            final_name = "{}/{}/{}/{}".format(filename.split("/")[0], folder, "5", filename.split("/")[-1])
            img = Image.fromarray(np.uint8(adv_samples[4][index]*255))
            img.save(final_name)
    return

In [29]:
from keras.utils import to_categorical
from PIL import Image

def get_data_generator(df, indices, for_training, batch_size=16):
    # filenames variable is not being passed right now for the evaluation but would be
    # required if we want to generate the dataset of adversarial images
    images, ages, races, genders, filenames = [], [], [], [], []
    while True:
        for i in indices:
            r = df.iloc[i]
            file, age, race, gender = r['file'], r['age'], r['race_id'], r['gender_id']
            im = Image.open(file)
            im = im.resize((IM_WIDTH, IM_HEIGHT))
            im = np.array(im) / 255.0
            # filenames.append(file)
            images.append(im)
            ages.append(age / max_age)
            races.append(to_categorical(race, len(RACE_ID_MAP)))
            genders.append(to_categorical(gender, 2))
            if len(images) >= batch_size:
                yield np.array(images), [np.array(ages), np.array(races), np.array(genders)] #, filenames
                images, ages, races, genders, filenames = [], [], [], [], []
        if not for_training:
            break

In [11]:
# random seed is not common with model training on the other notebook
p = np.random.permutation(len(df))
train_up_to = int(len(df) * TRAIN_TEST_SPLIT)
test_idx = p[train_up_to:]

df['gender_id'] = df['gender'].map(lambda gender: GENDER_ID_MAP[gender])
df['race_id'] = df['race'].map(lambda race: RACE_ID_MAP[race])

max_age = df['age'].max()

test_gen = get_data_generator(df, test_idx, for_training=False, batch_size=128)

In [12]:
model = load_model(adv_model_path)

W0923 08:42:15.419989 139653548660544 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0923 08:42:19.614129 139653548660544 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



In [13]:
save_all_adv_samples(test_gen, model)

## Evaluate clean model

In [21]:
def get_adv_data_generator(df, indices, eps, for_training=False, batch_size=128):
    images, ages, races, genders = [], [], [], []
    folder = "adv_generated_imgs"
    while True:
        for i in indices:
            r = df.iloc[i]
            file, age, race, gender = r['file'], r['age'], r['race_id'], r['gender_id']
            filename = "{}/{}/{}/{}".format(file.split("/")[0], folder, str(eps), file.split("/")[-1])
            im = Image.open(filename)
            im = im.resize((IM_WIDTH, IM_HEIGHT))
            im = np.array(im) / 255.0
            images.append(im)
            ages.append(age / max_age)
            races.append(to_categorical(race, len(RACE_ID_MAP)))
            genders.append(to_categorical(gender, 2))
            if len(images) >= batch_size:
                yield np.array(images), [np.array(ages), np.array(races), np.array(genders)]
                images, ages, races, genders = [], [], [], []
        if not for_training:
            break

In [15]:
# load clean model
model_path = "./models/VGG16_clean_model.h5"
model = load_model(model_path)

In [30]:
# eps = 0
test_gen = get_data_generator(df, test_idx, for_training=False, batch_size=128)
dict(zip(model.metrics_names, model.evaluate_generator(test_gen, steps=len(test_idx)//128)))

{u'age_output_loss': 0.014563413336873055,
 u'age_output_mae': 0.09220802783966064,
 u'gender_output_accuracy': 0.9751090407371521,
 u'gender_output_loss': 0.10134270042181015,
 'loss': 0.5926063060760498,
 u'race_output_accuracy': 0.8860828280448914,
 u'race_output_loss': 0.46940693259239197}

In [22]:
eps = 1
test_gen = get_adv_data_generator(df, test_idx, eps, for_training=False, batch_size=128)
dict(zip(model.metrics_names, model.evaluate_generator(test_gen, steps=len(test_idx)//128)))

{u'age_output_loss': 0.027471547946333885,
 u'age_output_mae': 0.13309116661548615,
 u'gender_output_accuracy': 0.8731831312179565,
 u'gender_output_loss': 0.3502197563648224,
 'loss': 2.1239891052246094,
 u'race_output_accuracy': 0.565588653087616,
 u'race_output_loss': 2.089019775390625}

In [23]:
eps = 2
test_gen = get_adv_data_generator(df, test_idx, eps, for_training=False, batch_size=128)
dict(zip(model.metrics_names, model.evaluate_generator(test_gen, steps=len(test_idx)//128)))

{u'age_output_loss': 0.03649305924773216,
 u'age_output_mae': 0.1581742763519287,
 u'gender_output_accuracy': 0.7738008499145508,
 u'gender_output_loss': 0.5750532746315002,
 'loss': 3.5959367752075195,
 u'race_output_accuracy': 0.4502180218696594,
 u'race_output_loss': 3.2963874340057373}

In [24]:
eps = 3
test_gen = get_adv_data_generator(df, test_idx, eps, for_training=False, batch_size=128)
dict(zip(model.metrics_names, model.evaluate_generator(test_gen, steps=len(test_idx)//128)))

{u'age_output_loss': 0.04659625515341759,
 u'age_output_mae': 0.1837593913078308,
 u'gender_output_accuracy': 0.6500726938247681,
 u'gender_output_loss': 0.8763740062713623,
 'loss': 4.840598106384277,
 u'race_output_accuracy': 0.4075218141078949,
 u'race_output_loss': 4.217748641967773}

In [25]:
eps = 4
test_gen = get_adv_data_generator(df, test_idx, eps, for_training=False, batch_size=128)
dict(zip(model.metrics_names, model.evaluate_generator(test_gen, steps=len(test_idx)//128)))

{u'age_output_loss': 0.05641218647360802,
 u'age_output_mae': 0.20550322532653809,
 u'gender_output_accuracy': 0.5803052186965942,
 u'gender_output_loss': 1.1167323589324951,
 'loss': 5.559659481048584,
 u'race_output_accuracy': 0.403888076543808,
 u'race_output_loss': 4.7010817527771}

In [26]:
eps = 5
test_gen = get_adv_data_generator(df, test_idx, eps, for_training=False, batch_size=128)
dict(zip(model.metrics_names, model.evaluate_generator(test_gen, steps=len(test_idx)//128)))

{u'age_output_loss': 0.062024276703596115,
 u'age_output_mae': 0.2172510027885437,
 u'gender_output_accuracy': 0.5432412624359131,
 u'gender_output_loss': 1.264703631401062,
 'loss': 5.984292030334473,
 u'race_output_accuracy': 0.4035246968269348,
 u'race_output_loss': 4.879197120666504}